# Compute pRF cortical magnification base on pRF parameters 

In [1]:
# Stop warnings
import warnings
warnings.filterwarnings("ignore")

# Import 
import os 
import time
import numpy as np
import pandas as pd
from scipy import stats
import neuropythy as ny
import matplotlib.pyplot as plt
import scipy.sparse.csgraph as cs
import gc

# personal import 
from plot_utils import plotly_template, prf_ecc_pcm_plot
from math_utils import weighted_regression, weighted_nan_percentile, weighted_nan_median



# Apply Wang atlas

In [2]:
def apply_wang(sub, h):
    """Returns the Wang atlas as a property for a subjects hemisphere.

    `apply_wang(sub, 'lh')` applies the atlas to the subject's left hemisphere.
    `apply_wang(sub, 'rh')` applies the atlas to the subject's right hemisphere.
    """
    atl = apply_wang.atlas[h]
    hem = sub.hemis[h]
    fsa = ny.freesurfer_subject(apply_wang.fsaverage_path)
    fsa = fsa.hemis[h]
    return fsa.interpolate(hem, atl)
apply_wang.atlas = {
    h: ny.load(os.path.join(ny.library_path(), 'data', 'fsaverage', 'surf', f'{h}.wang15_mplbl.v1_0.mgz'))
    for h in ('lh', 'rh')}
apply_wang.fsaverage_path = '/home/jovyan/shared/data/freesurfer/fsaverage'

# Geodesic distance (mm)

In [3]:
def compute_geodesic_distance(vert_of_interest_idx, adjacency_matrix, max_distance=None):
    """
    Compute geodesic distances from a vertex to all others in the mesh, optionally limited to max_distance.

    Parameters
    ----------
    vert_of_interest_idx : int
        Index of the vertex from which to compute distances.
    adjacency_matrix : scipy.sparse matrix
        Sparse matrix representing the adjacency of the mesh.
    max_distance : float or None, optional
        If provided, only return distances less than this value.

    Returns
    -------
    median_distance : float
        Median geodesic distance (within mask if max_distance is provided).
    vert_dist_idx : array of int, optional
        Indices of vertices within max_distance (only returned if max_distance is not None).
    """
    distances = cs.dijkstra(adjacency_matrix, indices=[vert_of_interest_idx], directed=False)[0]

    if max_distance is not None:
        distance_mask = distances < max_distance
        vert_dist_idx = np.where(distance_mask)[0]
        median_distance = np.median(distances[distance_mask])
        return median_distance, vert_dist_idx
    else:
        median_distance = np.median(distances)
        return median_distance
        

# Visual distance (dva)

In [4]:
def compute_pRF_distance(vert_of_interest_idx, roi_vertices_hemi_mask, prf_x, prf_y):
    
    dist_array = []
    for vert_num, vert_idx in enumerate(roi_vertices_hemi_mask):
        # Acces pRF position
        target_x = prf_x[vert_of_interest_idx]
        target_y = prf_y[vert_of_interest_idx]
        neigboor_x = prf_x[vert_idx]
        neigboor_y = prf_y[vert_idx]
    
        # Compute visual distance
        vert_prf_dist_array_vert = np.sqrt((target_x - neigboor_x)**2 + (target_y - neigboor_y)**2)
        dist_array.append(vert_prf_dist_array_vert)

    # compute the median of visual distance
    vert_prf_dist_array_median = np.median(dist_array)
    
    return vert_prf_dist_array_median
    

# Compute de pRF CM (mm/dva)

## Acces data

In [5]:
# Settings 
#subject_id = 118225
hemis = ['rh', 'lh']
rois = ["V1", "V2", "V3"]
roi_code = {
     "V1": 1,
     "V2": 2,
     "V3": 3}
max_ecc = 8
max_distance = 5

In [16]:
# Get first 10 subjects' numbers
allsub = ny.data['hcp_lines'].subject_list

In [7]:
# Load an HCP subject:
# sub = ny.hcp_subject(subject_id)
# data = []
# for ii in allsub[0:9]:
#     sub = ny.hcp_subject(ii)
#     data.append(sub)
# sub = ny.data['hcp_lines'].subjects[subject_id]
# wang_labels_lh = apply_wang(sub, 'lh')
# wang_labels_rh = apply_wang(sub, 'rh')

## pRF CM computation

In [14]:
group_sub = allsub[155:]
for n_sub, subject_id in enumerate(group_sub):
    start_time = time.time()
    print(f"{n_sub} / {len(group_sub)}")
    # Load an HCP subject:
    sub = ny.hcp_subject(subject_id)
    sub = ny.data['hcp_lines'].subjects[subject_id]
    prf_CM_df = pd.DataFrame()
    for n_hemi, hemi in enumerate(hemis): 
        for roi in rois : 
            # ROI mask et paramètres
            if hemi == 'lh':
                label_roi = sub.lh.prop('visual_area')
                prf_x_hemi = sub.lh.prop('prf_x')
                prf_y_hemi = sub.lh.prop('prf_y')
                prf_r2_hemi = sub.lh.prop('prf_variance_explained')
                prf_ecc_hemi = sub.lh.prop('prf_eccentricity')
                white = sub.lh.surface('white') 
            elif hemi == 'rh':
                label_roi = sub.rh.prop('visual_area')
                prf_x_hemi = sub.rh.prop('prf_x')
                prf_y_hemi = sub.rh.prop('prf_y')
                prf_r2_hemi = sub.rh.prop('prf_variance_explained')
                prf_ecc_hemi = sub.rh.prop('prf_eccentricity')
                white = sub.rh.surface('white')
           
            # Create the Roi mask
            roi_vertices_hemi_mask = np.where(label_roi == roi_code[roi])[0]
                    
            # Acces the adjacency_matrix of the roi
            submesh = white.submesh(roi_vertices_hemi_mask)
            adjacency_matrix = submesh.adjacency_matrix
           
        
            prf_dist_list = [] 
            geo_dist_list = []
            prf_ecc_list = []
            prf_r2_list = []
    
            for vert_num, vert_idx in enumerate(submesh.labels):
                vert_dist, vert_mask = compute_geodesic_distance(vert_num, adjacency_matrix, max_distance)
                geo_dist_list.append(vert_dist)
                prf_dist_list.append(compute_pRF_distance(vert_num, 
                                                          vert_mask, 
                                                          prf_x_hemi[roi_vertices_hemi_mask], 
                                                          prf_y_hemi[roi_vertices_hemi_mask]))
            
                prf_ecc_list.append(prf_ecc_hemi[vert_idx]) 
                prf_r2_list.append(prf_r2_hemi[vert_idx])
        
            prf_CM_df_hemi = pd.DataFrame()
            prf_CM_df_hemi['prf_dist'] = prf_dist_list
            prf_CM_df_hemi['geo_dist'] = geo_dist_list
            prf_CM_df_hemi['prf_ecc'] = prf_ecc_list
            prf_CM_df_hemi['prf_r2'] = prf_r2_list
            prf_CM_df_hemi['hemi'] = hemi  
            prf_CM_df_hemi['roi'] = roi
            prf_CM_df_hemi['subject'] = '{}'.format(subject_id)  
            
        
            prf_CM_df = pd.concat([prf_CM_df, prf_CM_df_hemi], ignore_index=True)

    prf_CM_df['pRF_CM'] = prf_CM_df['geo_dist'] / prf_CM_df['prf_dist']
    # Export DF 
    tsv_dir = '/home/jovyan/projects/pRF-project_NH2025/data/tsv'
    os.makedirs(tsv_dir, exist_ok=True)
    
    # tsv_fn = '{}/{}_prf_parameters.tsv'.format(tsv_dir, subject_id)
    tsv_fn = '{}/{}_prf_parameters_5mm.tsv'.format(tsv_dir, subject_id)
    
    prf_CM_df.to_csv(tsv_fn, sep="\t", na_rep='NaN', index=False)

    del sub
    del submesh
    
    
    end_time = time.time()
    print("Execution time: {:.2f} seconds".format(end_time - start_time))
    ny = ny.reload_neuropythy()

0 / 26
Execution time: 105.05 seconds
1 / 26
Execution time: 85.96 seconds
2 / 26
Execution time: 97.85 seconds
3 / 26
Execution time: 104.14 seconds
4 / 26
Execution time: 99.25 seconds
5 / 26
Execution time: 103.16 seconds
6 / 26
Execution time: 108.00 seconds
7 / 26
Execution time: 104.33 seconds
8 / 26
Execution time: 108.57 seconds
9 / 26
Execution time: 107.04 seconds
10 / 26
Execution time: 109.16 seconds
11 / 26
Execution time: 98.50 seconds
12 / 26
Execution time: 111.31 seconds
13 / 26
Execution time: 100.50 seconds
14 / 26
Execution time: 85.85 seconds
15 / 26
Execution time: 98.16 seconds
16 / 26
Execution time: 95.65 seconds
17 / 26
Execution time: 92.58 seconds
18 / 26
Execution time: 85.56 seconds
19 / 26
Execution time: 83.67 seconds
20 / 26
Execution time: 116.68 seconds
21 / 26
Execution time: 98.82 seconds
22 / 26
Execution time: 98.83 seconds
23 / 26
Execution time: 103.07 seconds
24 / 26
Execution time: 101.51 seconds
25 / 26
Execution time: 97.49 seconds
